<a href="https://colab.research.google.com/github/gitHubAndyLee2020/OpenAI_Gym_RL_Algorithms_Database/blob/main/DDPG_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DDPG

> About

- Consists of Actor and Critic, where the Actor generates some continuous action for given state, and Critic generates expected reward for some state and action
- Target Critic is used to stablish the training of Critic
- Critic is first trained using actual reward and future predicted reward using Target Critic as target value, and then the updated Critic is used to train the Actor by judging its performance

> Pro

- Simplicity and Ease of Implementation

> Con

- Sample Inefficiency

```
class Replay_buffer():
  def __init__(self, max_size=capacity):
    - Intialize the storage, maximum size, and pointer to keep track of current index

  def push(self, data):
    - If the length of the storage as reached the maximum size, start replacing the data from the oldest item in the storage
    - Otherwise, append the data to the storage

  def sample(self, batch_size):
    - Select random batch_size items from the storage and return them
```

```
class Actor(nn.Module):
  def __init__(self, state_dim, action_dim, max_action):
    - Map state input -> hidden layer -> action probability
    - Also stores the maximum value for continuous action in variable self.max_action

  def forward(self, x):
    - Feed the input through the neural network, and apply tanh to the output of the neural network before multiplying it with self.max_action. Since tanh squeezes values to be between -1 and 1, the range of the forward function output is +-maximum action value
```

```
class Critic(nn.Module):
  def __init__(self, state_dim, action_dim):
    - Map state input and action -> hidden layer -> expected reward value
  
  def forward(self, x, u):
    - Feed the x: state and u: action through the neural network
```

```
class DDPG(object):
  def __init__(self, state_dim, action_dim, max_action):
    - Initialize the actor and target actor networks, and copy the weights of actor network to target actor network. Initialize the actor network optimizer
    - Initialize the critic and target critic networks, and copy the weights of critic network to target critic network. Initialize the critic network optimizer
    - Initialize the replay buffer

  def select_action(self, state):
    - Feed the state to actor network and get the action value
    - Return the action value

  def update(self):
    - For the specified number of update iterations, run the following update loop
    - Update loop is as follows
    # Critic network update
    - 1. Fetch a sample from the replay buffer and unwrap the state, action,next state, done, reward. These values are tensors of length batch_size
    - 2. Compute the target Q value by feeding the state and action to the target critic network. The target Q value is the expected reward given some state and action. Then calculate the final target Q value using formula final target Q = reward + gamma * target Q value. This sets the target Q value that the critic network should aim to hit. The target critic network is used to stabilize training, as it avoids the target Q value from fluctuating alongside current Q value, leading to oscillation
    - 3. Compute the current Q value by feeding the state and action to critic network
    - 4. Calculate the loss between current Q value and target Q value
    - 5. Apply backpropagation to critic network
    # Actor network update
    - 1. Generate action tensor by feeding state tensor to actor network
    - 2. Feed the state tensor and generated action tensor to critic network to get the expected reward tensor
    - 3. Get the negated mean of the expected reward tensor to get the Actor loss. This essentially calculates how much reward is expected from the actions chosen by the actor network, where if the expected reward is large, the Actor loss is smaller (due to negation) and requires less adjustment, whereas if the expected reward is small, the Acto is bigger and requires larger adjustment
    - 4. Apply backpropagation
    - Afterwards, the target networks are partially updated from main networks

  def save(self):
    - Save the weights of the actor and critic networks

  def load(self):
    - Load the weights into the actor and critic networks
```

```
def main():
  - If it is test model, run the agent for some number of iterations to get the number of achieved steps for each iteration
  - Else if it is train model, for some number of iterations, collect trajectories, which is a tuple of state, next state, action, reward, done, and update the agent
```